<a href="https://colab.research.google.com/github/ashish899999/LSTM-PROJECT/blob/main/sentiment_analysis_using_LSTM_%26_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install kaggle

In [26]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection  import train_test_split
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [27]:
kaggle_dictionary=json.load(open("kaggle.json"))

In [28]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [29]:
import zipfile
zip_ref = zipfile.ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [30]:
data=pd.read_csv(r'/content/IMDB Dataset.csv')
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [31]:
data.shape

(50000, 2)

In [32]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [33]:
data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)

In [34]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [35]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=42)

In [36]:
train_data.shape

(40000, 2)

In [37]:
# tokenize text data
tokenizer=Tokenizer()
tokenizer.fit_on_texts(train_data['review'])
X_train=pad_sequences(tokenizer.texts_to_sequences(train_data['review']),maxlen=200)
X_test=pad_sequences(tokenizer.texts_to_sequences(test_data['review']),maxlen=200)


In [38]:
X_train

array([[  144,  1083,    16, ...,   205,   351,  3856],
       [  310,     5,   425, ...,    89,   103,     9],
       [    0,     0,     0, ...,     2,   710,    62],
       ...,
       [    0,     0,     0, ...,  1641,     2,   603],
       [    0,     0,     0, ...,   125,  7285, 16293],
       [    0,     0,     0, ...,    70,    73,  2062]], dtype=int32)

In [39]:
y_train=train_data['sentiment']
y_test=test_data['sentiment']

In [40]:
model=Sequential()

model.add(Embedding(input_dim=5000,output_dim=128,input_length=200))
model.add(LSTM(128, dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 128)          640000    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])


In [44]:
model.fit(X_train,y_train,epochs=10,batch_size=63,validation_split=0.2)

Epoch 1/10
508/508 [==============================] - 376s 739ms/step - loss: 0.3310 - accuracy: 0.8627 - val_loss: 0.2953 - val_accuracy: 0.8770
Epoch 2/10
508/508 [==============================] - 353s 694ms/step - loss: 0.2611 - accuracy: 0.8953 - val_loss: 0.3401 - val_accuracy: 0.8644
Epoch 3/10
508/508 [==============================] - 528s 1s/step - loss: 0.2192 - accuracy: 0.9138 - val_loss: 0.3149 - val_accuracy: 0.8777
Epoch 4/10
508/508 [==============================] - 366s 716ms/step - loss: 0.1865 - accuracy: 0.9284 - val_loss: 0.3523 - val_accuracy: 0.8756
Epoch 5/10
508/508 [==============================] - 350s 689ms/step - loss: 0.1650 - accuracy: 0.9374 - val_loss: 0.3781 - val_accuracy: 0.8692
Epoch 6/10
508/508 [==============================] - 344s 678ms/step - loss: 0.1415 - accuracy: 0.9478 - val_loss: 0.3668 - val_accuracy: 0.8723
Epoch 7/10
508/508 [==============================] - 348s 685ms/step - loss: 0.1271 - accuracy: 0.9531 - val_loss: 0.4029 - va

In [52]:
# building a predictive system
def predict_sentiment(review):
  # tokenization and pad review
  sequence=tokenizer.texts_to_sequences([review])
  padded_sequence=pad_sequences(sequence,maxlen=200)
  prediction=model.predict(padded_sequence)
  sentiment="Postive" if prediction[0][0]>0.5 else "Negative"
  return sentiment

In [53]:
new_review="movie was fantastic i loved it  "
sentiment=predict_sentiment(new_review)
print(sentiment)

1/1 [==============================] - 0s 64ms/step
Postive


In [54]:
new_review="movie was bad "
sentiment=predict_sentiment(new_review)
print(sentiment)

1/1 [==============================] - 0s 69ms/step
Negative


In [55]:
new_review="I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air con..."
sentiment=predict_sentiment(new_review)
print(sentiment)

1/1 [==============================] - 0s 64ms/step
Postive


In [56]:
new_review="Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his par..."
sentiment=predict_sentiment(new_review)
print(sentiment)

1/1 [==============================] - 0s 93ms/step
Negative
